In [3]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import torch
import pandas as pd 

In [39]:
# Assumes file is run from testing directory
FILE_PATH = "~/sysml/testing/model_test/yolov5-on-rpi4-2020/johnston_yolov5/yolov5/runs/detect"


def get_predictions_list(file_location, num_files):
    predictions_list = []
    # Loop through the files containing labels + bboxes for each frame
    for i in range(1, num_files+1):
        fname = file_location + "_" + str(i) + ".txt"
        # Read data
        df = pd.read_csv(fname, sep=' ', header=None)
        cols = ['label', 'x-center', 'y-center', 'width', 'height', 'conf-score']
        # cols = ['label', 'x-center', 'y-center', 'width', 'height'] #for ground truth

        df.columns = cols
        
        # Create FloatTensor containing boxes
        bbox_cols = df[['x-center', 'y-center', 'width', 'height']]
        boxes = torch.tensor(bbox_cols.values, dtype=torch.float32)
        
        # Create IntTensor containing labels
        labels = torch.tensor(df['label'])
        
        # Create FloatTensor containing scores
        scores = torch.tensor(df['conf-score'])
        
        # Append dict with boxes, labels, and scores to the list
        frame_dict = {'boxes':boxes,
                      'labels':labels,
                      'scores':scores
                      }
        predictions_list.append(frame_dict)
        
        #if (i%50 == 0):
            #print(predictions_list)
        
    return predictions_list
        

def calculate_accuracy(ground_truth, prediction):
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(prediction, ground_truth)
    result = metric.compute()
    return result['map'].item()



    #mAP = calculate_accuracy(ground_truth, predictions)
    #print("mAP: ", mAP)


In [37]:
ground_truth = get_predictions_list(FILE_PATH + "/german_ground_truth/labels/german", 200)

In [40]:
len(ground_truth)

200

In [41]:
pred = get_predictions_list(FILE_PATH + '/sparse_yolov5l_gt/labels/sparse', 200)

In [42]:
calculate_accuracy(ground_truth, pred)

0.0

In [ ]:
print("hello adam")